In [1]:
import numpy as np

In [2]:
def diffsize(x, y):
    return np.sqrt(np.sum((x-y)**2))

vdiffsize = np.vectorize(diffsize, signature="(n),(n)->()")

In [3]:
x = np.random.rand(10, 3)
y = np.random.rand(10, 3)

In [4]:
for i in range(x.shape[0]):
    print(diffsize(x[i,:], y[i,:]))

0.8042900952586733
0.5179234454291365
0.46810367849412865
0.45295014904531267
1.412724884089182
0.7678116867250895
0.6347081138730593
0.4202205012479076
0.6832359412149869
0.7138988140284894


In [5]:
vdiffsize(x,y)

array([0.8042901 , 0.51792345, 0.46810368, 0.45295015, 1.41272488,
       0.76781169, 0.63470811, 0.4202205 , 0.68323594, 0.71389881])

# 行列化

`m[i,j] = f(x[i,:], y[j,:])` となるような行列を作る

In [9]:
def f(x, y):
    return x[0] + y[0]/10

dim = 2
numx = 3
numy = 4
vf = np.vectorize(f, signature="(n),(n)->()")
xs, ys = np.zeros((numx,dim)), np.zeros((numy,dim))
xs[:,0] = np.arange(numx)
ys[:,0] = np.arange(numy)
print(xs)
print(ys)

[[0. 0.]
 [1. 0.]
 [2. 0.]]
[[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]]


In [14]:
# step1.
# ブロードキャストによって、必要な要素数になるまでコピーする.
# この時、最後の2つの次元が、xs,ysに合うように次元を調整する.
xs2, ys2 = np.zeros((numy,numx,dim)), np.zeros((numx,numy,dim))
xs2[:,:,:] = xs
ys2[:,:,:] = ys
# step2.
# xs2, ys2が同じ形になるように、転地によって次元を調整する.
xs2 = xs2.transpose((1,0,2))
vf(xs2, ys2)

array([[0. , 0.1, 0.2, 0.3],
       [1. , 1.1, 1.2, 1.3],
       [2. , 2.1, 2.2, 2.3]])